In [12]:
print('Importing packages...')
import os
import pandas
import numpy as np
import time
from matplotlib import pyplot as plt
import requests
import json
from pprint import pprint
from IPython.display import display, HTML
pandas.set_option('display.max_colwidth', None)

current_year = 2022

apikey = '92801b058cccd057649dfe29d55d185387d1363a'

thisdir = '/home/idies/workspace/21cc/raddick/community_reinvestment_act/'
data_dir = thisdir + 'city_data/'
# loans_dir = data_dir + 'Community Reinvestment Act/'
# jobs_dir = data_dir + 'LODES8/'
census_homedir = '/home/idies/workspace/21cc/Data/Census/ACS5/{0:.0f}/'.format(current_year)
census_data_dir = census_homedir + 'data/'
census_metadata_dir = census_homedir + 'metadata/'

figdir = thisdir + 'figures/'

baltimore_shapefile_basedir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'
census_shapefile_tiger_basedir = '/home/idies/workspace/21cc/Data/Census/Shapefiles/TIGER/'

print('ok')

Importing packages...
ok


In [148]:
# Baltimore: place = 04000, state = 24

#https://api.census.gov/data/2022/acs/acs5?get=NAME,B01001_001E,B02001_003E&for=place:04000&in=state:24
# Total population: B01001_001
# Median home value: B25077_001
# Population in owner-occupied housing units: B25008_002
### Total population in housing units: B25008_001
# Median family income: B19113_001
# Population with income below poverty level: B17101_002
### Total population for whom poverty status determined: B17101_001
# Population with bachelor degree or higher: B15001_009 + B15001_010
### Total population with educational attainment determined: B15001_001
# Population unemployed (16 and over): B23025_005
### Total population with known employment status: B23025_001
# Population black alone: B02001_003
### Total population whose race is known: B02001_001

print('Looking up via census API...')

print('loading variables...')
thevars = []
thevars += ['B01001_001', 'B25077_001', 'B25008_001', 'B25008_002', 'B19113_001']
thevars += ['B17101_002', 'B17101_001', 'B15003_022', 'B15003_023', 'B15003_024', 'B15003_025', 'B15003_001']
thevars += ['B23025_005', 'B23025_001', 'B02001_003', 'B02001_001']

print('loading place names...')
placenames_df = pandas.DataFrame(data=None, columns=['city', 'place', 'state']).set_index('city')

placenames_df.loc['National'] = [np.nan, np.nan]
placenames_df.loc['Baltimore'] = ['04000', '24']
placenames_df.loc['Cleveland'] = ['16000', '39']
placenames_df.loc['Detroit'] = ['22000', '26']
placenames_df.loc['Newark'] = ['51000', '34']
placenames_df.loc['Philadelphia'] = ['60000', '42']
placenames_df.loc['Pittsburgh'] = ['61000', '42']
placenames_df.loc['Richmond'] = ['67000', '51']
placenames_df.loc['St. Louis'] = ['65000', '29']
#placenames_df.loc['Washington, DC'] = ['50000', '11']
placenames_df.loc['Washington'] = ['50000', '11']


print('getting estimates from census API...')
table1_data_df = pandas.DataFrame(data=None, columns=['city']+thevars).set_index('city')


for thiscity, thisrow in placenames_df[placenames_df.index == 'National'].iterrows():
    print('\t{0:}...'.format(thiscity))
    call = 'https://api.census.gov/data/{0:.0f}/acs/acs5?get='.format(current_year)
    for x in thevars[:-1]:
        call += x + 'E,'
    call += thevars[-1] + 'E'
    call += '&for=us:*'
    call += '&key={0:}'.format(apikey)
    #print(call)
    
    r = requests.get(call)
    resp = json.loads(r.text)
    table1_data_df.loc[thiscity] = resp[1][:-1]

for thiscity, thisrow in placenames_df[placenames_df.index != 'National'].iterrows():
    print('\t{0:}...'.format(thiscity))
    call = 'https://api.census.gov/data/{0:.0f}/acs/acs5?get='.format(current_year)
    for x in thevars[:-1]:
        call += x + 'E,'
    call += thevars[-1] + 'E'
    call += '&for=place:{0:}&&in=state:{1:}'.format(thisrow['place'], thisrow['state'])
    call += '&key={0:}'.format(apikey)
    #print(call)
    
    r = requests.get(call)
    resp = json.loads(r.text)
    table1_data_df.loc[thiscity] = resp[1][:-2]
    
final_columns = ['City']
final_columns += ['Total population', 'Median home value', 'Home ownership rate', 'Median family income']
final_columns += ['Poverty rate', 'Share of people with a bachelors degree or higher']
final_columns += ['Unemployment rate', 'Percent black']


print('calculating summary variables...')
table1_df = pandas.DataFrame(data=None, columns=final_columns).set_index('City')

table1_df.loc[:, 'Total population'] = pandas.to_numeric(table1_data_df['B01001_001'], errors='coerce')
table1_df.loc[:, 'Median home value'] = pandas.to_numeric(table1_data_df['B25077_001'], errors='coerce')
table1_df.loc[:, 'Home ownership rate'] = pandas.to_numeric(table1_data_df['B25008_002'], errors='coerce') / pandas.to_numeric(table1_data_df['B25008_001'], errors='coerce')
table1_df.loc[:, 'Median family income'] = pandas.to_numeric(table1_data_df['B19113_001'], errors='coerce')
table1_df.loc[:, 'Poverty rate'] = pandas.to_numeric(table1_data_df['B17101_002'], errors='coerce') / pandas.to_numeric(table1_data_df['B17101_001'], errors='coerce')
table1_df.loc[:, 'Share of people with a bachelors degree or higher'] = (pandas.to_numeric(table1_data_df['B15003_022'], errors='coerce') + pandas.to_numeric(table1_data_df['B15003_023'], errors='coerce') + pandas.to_numeric(table1_data_df['B15003_024'], errors='coerce') + pandas.to_numeric(table1_data_df['B15003_025'], errors='coerce')) / pandas.to_numeric(table1_data_df['B15003_001'], errors='coerce')
table1_df.loc[:, 'Unemployment rate'] = pandas.to_numeric(table1_data_df['B23025_005'], errors='coerce') / pandas.to_numeric(table1_data_df['B23025_001'], errors='coerce')
table1_df.loc[:, 'Percent black'] = pandas.to_numeric(table1_data_df['B02001_003'], errors='coerce') / pandas.to_numeric(table1_data_df['B02001_001'], errors='coerce')

print('building table 1...')
thehtml = '<p><strong>Table 1</strong></p>'
thehtml += '<table>'
thehtml += '<tr>'
thehtml += '<th></th>'
for thiscol in table1_df.columns.tolist():
    thehtml += '<th>{0:}</th>'.format(thiscol)
for ix, thisrow in table1_df.iterrows():
    thehtml += '<tr>'
    thehtml += '<th>{0:}</th>'.format(ix)
    thehtml += '<td>{0:,.0f}</td>'.format(thisrow['Total population'])
    thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median home value'])
    thehtml += '<td>{0:.0%}</td>'.format(thisrow['Home ownership rate'])
    thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median family income'])
    thehtml += '<td>{0:.0%}</td>'.format(thisrow['Poverty rate'])
    thehtml += '<td>{0:.0%}</td>'.format(thisrow['Share of people with a bachelors degree or higher'])
    thehtml += '<td>{0:.1%}</td>'.format(thisrow['Unemployment rate'])
    thehtml += '<td>{0:.0%}</td>'.format(thisrow['Percent black'])
    
display(HTML(thehtml))


Looking up via census API...
loading variables...
loading place names...
getting estimates from census API...
	National...
	Baltimore...
	Cleveland...
	Detroit...
	Newark...
	Philadelphia...
	Pittsburgh...
	Richmond...
	St. Louis...
	Washington...
calculating summary variables...
building table 1...


,Total population,Median home value,Home ownership rate,Median family income,Poverty rate,Share of people with a bachelors degree or higher,Unemployment rate,Percent black
National,"331,097,593","$281,900",67%,"$92,646",12%,34%,3.4%,12%
Baltimore,"584,548","$202,900",52%,"$72,972",19%,35%,4.3%,61%
Cleveland,"370,365","$87,400",44%,"$46,784",31%,20%,7.1%,47%
Detroit,"636,787","$66,700",51%,"$46,424",31%,17%,8.0%,78%
Newark,"307,355","$312,300",27%,"$56,099",24%,17%,6.7%,47%
Philadelphia,"1,593,208","$215,500",56%,"$67,168",22%,34%,5.4%,40%
Pittsburgh,"303,843","$171,800",51%,"$85,816",19%,47%,3.6%,23%
Richmond,"227,171","$308,300",45%,"$79,453",19%,44%,4.1%,44%
St. Louis,"298,018","$174,100",50%,"$72,655",20%,39%,3.6%,44%
Washington,"670,587","$705,000",45%,"$142,328",14%,63%,5.1%,44%


In [146]:


# # Baltimore: place = 04000, state = 24

# #https://api.census.gov/data/2022/acs/acs5?get=NAME,B01001_001E,B02001_003E&for=place:04000&in=state:24
# # Total population: B01001_001
# # Median home value: B25077_001
# # Population in owner-occupied housing units: B25008_002
# ### Total population in housing units: B25008_001
# # Median family income: B19113_001
# # Population with income below poverty level: B17101_002
# ### Total population for whom poverty status determined: B17101_001
# # Population with bachelor degree or higher: B15001_009 + B15001_010
# ### Total population with educational attainment determined: B15001_001
# # Population unemployed (16 and over): B23025_005
# ### Total population with known employment status: B23025_001
# # Population black alone: B02001_003
# ### Total population whose race is known: B02001_001

# print('Looking up via census API...')

# print('loading variables...')
# thevars = []
# thevars += ['B01001_001', 'B25077_001', 'B25008_001', 'B25008_002', 'B19113_001']
# thevars += ['B17101_002', 'B17101_001', 'B15003_022', 'B15003_023', 'B15003_024', 'B15003_025', 'B15003_001']
# thevars += ['B23025_005', 'B23025_001', 'B02001_003', 'B02001_001']

# print('loading place names...')
# placenames_df = pandas.DataFrame(data=None, columns=['city', 'place', 'state']).set_index('city')

# placenames_df.loc['National'] = [np.nan, np.nan]
# placenames_df.loc['Baltimore'] = ['04000', '24']
# placenames_df.loc['Cleveland'] = ['16000', '39']
# placenames_df.loc['Detroit'] = ['22000', '26']
# placenames_df.loc['Newark'] = ['51000', '34']
# placenames_df.loc['Philadelphia'] = ['60000', '42']
# placenames_df.loc['Pittsburgh'] = ['61000', '42']
# placenames_df.loc['Richmond'] = ['67000', '51']
# placenames_df.loc['St. Louis'] = ['65000', '29']
# #placenames_df.loc['Washington, DC'] = ['50000', '11']
# placenames_df.loc['Washington'] = ['50000', '11']


# print('getting estimates from census API...')
# table1_data_df = pandas.DataFrame(data=None, columns=['city']+thevars).set_index('city')


# for thiscity, thisrow in placenames_df[placenames_df.index == 'National'].iterrows():
#     print('\t{0:}...'.format(thiscity))
#     call = 'https://api.census.gov/data/{0:.0f}/acs/acs5?get='.format(current_year)
#     for x in thevars[:-1]:
#         call += x + 'E,'
#     call += thevars[-1] + 'E'
#     call += '&for=us:*'
#     call += '&key={0:}'.format(apikey)
#     #print(call)
    
#     r = requests.get(call)
#     resp = json.loads(r.text)
#     table1_data_df.loc[thiscity] = resp[1][:-1]

# for thiscity, thisrow in placenames_df[placenames_df.index != 'National'].iterrows():
#     print('\t{0:}...'.format(thiscity))
#     call = 'https://api.census.gov/data/{0:.0f}/acs/acs5?get='.format(current_year)
#     for x in thevars[:-1]:
#         call += x + 'E,'
#     call += thevars[-1] + 'E'
#     call += '&for=place:{0:}&&in=state:{1:}'.format(thisrow['place'], thisrow['state'])
#     call += '&key={0:}'.format(apikey)
#     #print(call)
    
#     r = requests.get(call)
#     resp = json.loads(r.text)
#     table1_data_df.loc[thiscity] = resp[1][:-2]
    
# final_columns = ['City']
# final_columns += ['Total population', 'Median home value', 'Home ownership rate', 'Median family income']
# final_columns += ['Poverty rate', 'Share of people with a bachelors degree or higher']
# final_columns += ['Unemployment rate', 'Percent black']


# print('calculating summary variables...')
# table1_df = pandas.DataFrame(data=None, columns=final_columns).set_index('City')

# table1_df.loc[:, 'Total population'] = pandas.to_numeric(table1_data_df['B01001_001'], errors='coerce')
# table1_df.loc[:, 'Median home value'] = pandas.to_numeric(table1_data_df['B25077_001'], errors='coerce')
# table1_df.loc[:, 'Home ownership rate'] = pandas.to_numeric(table1_data_df['B25008_002'], errors='coerce') / pandas.to_numeric(table1_data_df['B25008_001'], errors='coerce')
# table1_df.loc[:, 'Median family income'] = pandas.to_numeric(table1_data_df['B19113_001'], errors='coerce')
# table1_df.loc[:, 'Poverty rate'] = pandas.to_numeric(table1_data_df['B17101_002'], errors='coerce') / pandas.to_numeric(table1_data_df['B17101_001'], errors='coerce')
# table1_df.loc[:, 'Share of people with a bachelors degree or higher'] = (pandas.to_numeric(table1_data_df['B15003_022'], errors='coerce') + pandas.to_numeric(table1_data_df['B15003_023'], errors='coerce') + pandas.to_numeric(table1_data_df['B15003_024'], errors='coerce') + pandas.to_numeric(table1_data_df['B15003_025'], errors='coerce')) / pandas.to_numeric(table1_data_df['B15003_001'], errors='coerce')
# table1_df.loc[:, 'Unemployment rate'] = pandas.to_numeric(table1_data_df['B23025_005'], errors='coerce') / pandas.to_numeric(table1_data_df['B23025_001'], errors='coerce')
# table1_df.loc[:, 'Percent black'] = pandas.to_numeric(table1_data_df['B02001_003'], errors='coerce') / pandas.to_numeric(table1_data_df['B02001_001'], errors='coerce')

# print('building table 1...')
# thehtml = '<p><strong>Table 1 calculated from place (level 160) in Census API</strong></p>'
# thehtml += '<table>'
# thehtml += '<tr>'
# thehtml += '<th></th>'
# for thiscol in table1_df.columns.tolist():
#     thehtml += '<th>{0:}</th>'.format(thiscol)
# for ix, thisrow in table1_df.iterrows():
#     thehtml += '<tr>'
#     thehtml += '<th>{0:}</th>'.format(ix)
#     thehtml += '<td>{0:,.0f}</td>'.format(thisrow['Total population'])
#     thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median home value'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Home ownership rate'])
#     thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median family income'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Poverty rate'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Share of people with a bachelors degree or higher'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Unemployment rate'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Percent black'])
    
# display(HTML(thehtml))


# print('getting margins of error from census API...')

# print('loading variables...')
# thevars = []
# thevars += ['B01001_001', 'B25077_001', 'B25008_001', 'B25008_002', 'B19113_001']
# thevars += ['B17101_002', 'B17101_001', 'B15003_022', 'B15003_023', 'B15003_024', 'B15003_025', 'B15003_001']
# thevars += ['B23025_005', 'B23025_001', 'B02001_003', 'B02001_001']

# table1a_data_df = pandas.DataFrame(data=None, columns=['city']+thevars).set_index('city')

# for thiscity, thisrow in placenames_df[placenames_df.index == 'National'].iterrows():
#     print('\t{0:}...'.format(thiscity))
#     call = 'https://api.census.gov/data/{0:.0f}/acs/acs5?get='.format(current_year)
#     for x in thevars[:-1]:
#         call += x + 'M,'
#     call += thevars[-1] + 'M'
#     call += '&for=us:*'
#     call += '&key={0:}'.format(apikey)
#     #print(call)
    
#     r = requests.get(call)
#     resp = json.loads(r.text)
#     table1a_data_df.loc[thiscity] = resp[1][:-1]

# for thiscity, thisrow in placenames_df[placenames_df.index != 'National'].iterrows():
#     print('\t{0:}...'.format(thiscity))
#     call = 'https://api.census.gov/data/{0:.0f}/acs/acs5?get='.format(current_year)
#     for x in thevars[:-1]:
#         call += x + 'M,'
#     call += thevars[-1] + 'M'
#     call += '&for=place:{0:}&&in=state:{1:}'.format(thisrow['place'], thisrow['state'])
#     call += '&key={0:}'.format(apikey)
#     #print(call)
    
#     r = requests.get(call)
#     resp = json.loads(r.text)
#     table1a_data_df.loc[thiscity] = resp[1][:-2]
    
# final_columns = ['City']
# final_columns += ['Total population', 'Median home value', 'Home ownership rate', 'Median family income']
# final_columns += ['Poverty rate', 'Share of people with a bachelors degree or higher']
# final_columns += ['Unemployment rate', 'Percent black']


# print('calculating summary variables...')
# table1a_df = pandas.DataFrame(data=None, columns=['City']+final_columns).set_index('City')

# table1a_df.loc[:, 'Total population'] = pandas.to_numeric(table1a_data_df['B01001_001'], errors='coerce')
# table1a_df.loc[:, 'Median home value'] = pandas.to_numeric(table1a_data_df['B25077_001'], errors='coerce')
# table1a_df.loc[:, 'Home ownership rate'] = pandas.to_numeric(table1a_data_df['B25008_002'], errors='coerce') / pandas.to_numeric(table1a_data_df['B25008_001'], errors='coerce')
# table1a_df.loc[:, 'Median family income'] = pandas.to_numeric(table1a_data_df['B19113_001'], errors='coerce')
# table1a_df.loc[:, 'Poverty rate'] = pandas.to_numeric(table1a_data_df['B17101_002'], errors='coerce') / pandas.to_numeric(table1a_data_df['B17101_001'], errors='coerce')
# table1a_df.loc[:, 'Share of people with a bachelors degree or higher'] = (pandas.to_numeric(table1a_data_df['B15003_022'], errors='coerce') + pandas.to_numeric(table1a_data_df['B15003_023'], errors='coerce') + pandas.to_numeric(table1a_data_df['B15003_024'], errors='coerce') + pandas.to_numeric(table1a_data_df['B15003_025'], errors='coerce')) / pandas.to_numeric(table1a_data_df['B15003_001'], errors='coerce')
# table1a_df.loc[:, 'Unemployment rate'] = pandas.to_numeric(table1a_data_df['B23025_005'], errors='coerce') / pandas.to_numeric(table1a_data_df['B23025_001'], errors='coerce')
# table1a_df.loc[:, 'Percent black'] = pandas.to_numeric(table1a_data_df['B02001_003'], errors='coerce') / pandas.to_numeric(table1a_data_df['B02001_001'], errors='coerce')

# print('building table 1a...')
# thehtml = '<p><strong>Table 1a: margins of error calculated from place (level 160) in Census API</strong></p>'
# thehtml += '<table>'
# thehtml += '<tr>'
# thehtml += '<th></th>'
# for thiscol in table1a_df.columns.tolist():
#     thehtml += '<th>{0:}</th>'.format(thiscol)
# for ix, thisrow in table1a_df.iterrows():
#     thehtml += '<tr>'
#     thehtml += '<th>{0:}</th>'.format(ix)
#     if (thisrow['Total population'] < 0):
#         thehtml += '<td>-</td>';
#     else:
#         thehtml += '<td>{0:,.0f}</td>'.format(thisrow['Total population'])
#     thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median home value'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Home ownership rate'])
#     thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median family income'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Poverty rate'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Share of people with a bachelors degree or higher'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Unemployment rate'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Percent black'])
    
# display(HTML(thehtml))



# print('Calculating from our data...')
# filenames = sorted([x for x in os.listdir(data_dir) if x[-4:] == '.csv'])

# final_columns = ['City']
# final_columns += ['Total population', 'Median home value', 'Home ownership rate', 'Median family income']
# final_columns += ['Poverty rate', 'Share of people with a bachelors degree or higher']
# final_columns += ['Unemployment rate', 'Percent black']


# print('calculating summary variables...')
# table1b_df = pandas.DataFrame(data=None, columns=final_columns).set_index('City')

# for thisfilename in filenames[0:2]:
#     thiscity = thisfilename.split('_')[0]
#     print('\t{0:}...'.format(thiscity.title()))
#     df = pandas.read_csv(data_dir+thisfilename, low_memory=False, index_col=['GEOID', 'year'])
#     demog_df = df.xs(current_year, level='year')

#     table1b_df.loc[thiscity.title(), 'Total population'] = demog_df['pop_total'].sum()

#     weighted_avg_median_home_value = 0
#     for ix, thisrow in demog_df[demog_df['median_home_value'].notnull()].iterrows():
#         weighted_avg_median_home_value += (thisrow['pop_total'] / demog_df['pop_total'].sum()) * thisrow['median_home_value']
#     table1b_df.loc[thiscity.title(), 'Median home value'] = weighted_avg_median_home_value
    
#     weighted_avg_mfi = 0
#     for ix, thisrow in demog_df[demog_df['mfi'].notnull()].iterrows():
#         weighted_avg_mfi += (thisrow['pop_total'] / demog_df['pop_total'].sum()) * thisrow['mfi']
#     table1b_df.loc[thiscity.title(), 'Median family income'] = weighted_avg_mfi
        
#     weighted_poverty = 0
#     for ix, thisrow in demog_df[(demog_df['poverty_past_12_months'].notnull()) & (demog_df['poverty_status_known'].notnull())].iterrows():
#         weighted_poverty += (thisrow['poverty_past_12_months'])
#     table1b_df.loc[thiscity.title(), 'Poverty rate'] = weighted_poverty / demog_df[(demog_df['poverty_past_12_months'].notnull()) & (demog_df['poverty_status_known'].notnull())]['poverty_status_known'].sum()

#     weighted_education = 0
#     for ix, thisrow in demog_df[(demog_df['educated'].notnull()) & (demog_df['pop_25plus'].notnull())].iterrows():
#         weighted_education += (thisrow['educated'])
#     table1b_df.loc[thiscity.title(), 'Share of people with a bachelors degree or higher'] = weighted_education / demog_df[(demog_df['educated'].notnull()) & (demog_df['pop_25plus'].notnull())]['pop_25plus'].sum()
    
#     weighted_unemployment = 0
#     for ix, thisrow in demog_df[(demog_df['unemployed_16plus'].notnull()) & (demog_df['labor_force_16plus'].notnull())].iterrows():
#         weighted_unemployment += (thisrow['unemployed_16plus'])
#     table1b_df.loc[thiscity.title(), 'Unemployment rate'] = weighted_unemployment / demog_df[(demog_df['unemployed_16plus'].notnull()) & (demog_df['labor_force_16plus'].notnull())]['labor_force_16plus'].sum()

#     weighted_black = 0
#     for ix, thisrow in demog_df[(demog_df['pop_black'].notnull())].iterrows():
#         weighted_black += (thisrow['pop_black'])
#     table1b_df.loc[thiscity.title(), 'Percent black'] = weighted_black / demog_df['pop_total'].sum()
    
    
# print('building table 1b...')
# thehtml = '<p><strong>Table 1b: calculated from our tract-based data</strong></p>'
# thehtml += '<table>'
# thehtml += '<tr>'
# thehtml += '<th></th>'
# for thiscol in table1b_df.columns.tolist():
#     thehtml += '<th>{0:}</th>'.format(thiscol)
# for ix, thisrow in table1b_df.iterrows():
#     thehtml += '<tr>'
#     thehtml += '<th>{0:}</th>'.format(ix)
#     thehtml += '<td>{0:,.0f}</td>'.format(thisrow['Total population'])
#     thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median home value'])
#     thehtml += '<td>-</td>'
#     thehtml += '<td>${0:,.0f}</td>'.format(thisrow['Median family income'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Poverty rate'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Share of people with a bachelors degree or higher'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Unemployment rate'])
#     thehtml += '<td>{0:.0%}</td>'.format(thisrow['Percent black'])
    
# display(HTML(thehtml))


# print('Finding discrepancies...')
# table1c_df = pandas.DataFrame(data=None, columns=table1b_df.reset_index().columns).set_index('City')

# for ix in table1b_df.index.tolist():
# #     print(table1_df.loc[ix])
# #     print(table1b_df.loc[ix])
#     for thiscol in table1c_df.columns:
#         table1c_df.loc[ix, thiscol] = table1_df.loc[ix][thiscol] - table1b_df.loc[ix][thiscol]
# #table1_df.index
# table1c_df

# print('building table 1c...')
# thehtml = '<p><strong>Table 1c: discrepancy</strong></p>'
# thehtml += '<table>'
# thehtml += '<tr>'
# thehtml += '<th></th>'
# for thiscol in table1b_df.columns.tolist():
#     thehtml += '<th>{0:}</th>'.format(thiscol)
# for ix, thisrow in table1c_df.iterrows():
#     thehtml += '<tr>'
#     thehtml += '<th>{0:}</th>'.format(ix)
#     thehtml += '<td>{0:,.0f} ({1:.1%})</td>'.format(thisrow['Total population'], thisrow['Total population'] / table1_df.loc[ix]['Total population'])
#     thehtml += '<td>${0:,.0f} ({1:.1%})</td>'.format(thisrow['Median home value'], thisrow['Median home value'] / table1_df.loc[ix]['Median home value'])
#     thehtml += '<td>-</td>'
#     thehtml += '<td>${0:,.0f} ({1:.1%})</td>'.format(thisrow['Median family income'], thisrow['Median family income'] / table1_df.loc[ix]['Median family income'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Poverty rate'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Share of people with a bachelors degree or higher'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Unemployment rate'], thisrow['Unemployment rate'])
#     thehtml += '<td>{0:.1%}</td>'.format(thisrow['Percent black'], thisrow['Percent black'])
    
# display(HTML(thehtml))


Looking up via census API...
loading variables...
loading place names...
getting estimates from census API...
	National...
	Baltimore...
	Cleveland...
	Detroit...
	Newark...
	Philadelphia...
	Pittsburgh...
	Richmond...
	St. Louis...
	Washington...
calculating summary variables...
building table 1...


,Total population,Median home value,Home ownership rate,Median family income,Poverty rate,Share of people with a bachelors degree or higher,Unemployment rate,Percent black
National,"331,097,593","$281,900",67%,"$92,646",12%,34%,3.4%,12%
Baltimore,"584,548","$202,900",52%,"$72,972",19%,35%,4.3%,61%
Cleveland,"370,365","$87,400",44%,"$46,784",31%,20%,7.1%,47%
Detroit,"636,787","$66,700",51%,"$46,424",31%,17%,8.0%,78%
Newark,"307,355","$312,300",27%,"$56,099",24%,17%,6.7%,47%
Philadelphia,"1,593,208","$215,500",56%,"$67,168",22%,34%,5.4%,40%
Pittsburgh,"303,843","$171,800",51%,"$85,816",19%,47%,3.6%,23%
Richmond,"227,171","$308,300",45%,"$79,453",19%,44%,4.1%,44%
St. Louis,"298,018","$174,100",50%,"$72,655",20%,39%,3.6%,44%
Washington,"670,587","$705,000",45%,"$142,328",14%,63%,5.1%,44%


getting margins of error from census API...
loading variables...
	National...
	Baltimore...
	Cleveland...
	Detroit...
	Newark...
	Philadelphia...
	Pittsburgh...
	Richmond...
	St. Louis...
	Washington...
calculating summary variables...
building table 1a...


,Total population,Median home value,Home ownership rate,Median family income,Poverty rate,Share of people with a bachelors degree or higher,Unemployment rate,Percent black
National,-,$221,-0%,$260,1870%,3353%,188.8%,-0%
Baltimore,-,"$3,756",1927%,"$1,951",829%,10096%,246.8%,-0%
Cleveland,56,"$1,985",1099%,"$1,162",906%,199%,93.3%,5168%
Detroit,125,"$1,448",1803%,"$1,100",1108%,183%,90.6%,2757%
Newark,84,"$10,451",1053%,"$3,175",635%,139%,61.2%,3692%
Philadelphia,-,"$2,089",1838%,"$1,394",922%,7405%,290.5%,-0%
Pittsburgh,92,"$3,718",962%,"$2,393",723%,279%,64.5%,2504%
Richmond,-,"$9,855",1339%,"$2,613",574%,2882%,235.2%,-0%
St. Louis,-,"$4,165",1899%,"$2,080",697%,4738%,225.9%,-0%
Washington,-,"$10,380",-0%,"$4,915",767%,5458%,325.9%,-0%


Calculating from our data...
calculating summary variables...
	Baltimore...
	Washington...
building table 1b...


,Total population,Median home value,Home ownership rate,Median family income,Poverty rate,Share of people with a bachelors degree or higher,Unemployment rate,Percent black
Baltimore,"581,125","$213,129",-,"$84,783",19%,35%,6.9%,61%
Washington,"670,587","$705,694",-,"$147,663",15%,63%,7.0%,44%


Finding discrepancies...
building table 1c...


,Total population,Median home value,Home ownership rate,Median family income,Poverty rate,Share of people with a bachelors degree or higher,Unemployment rate,Percent black
Baltimore,"3,423 (0.6%)","$-10,229 (-5.0%)",-,"$-11,811 (-16.2%)",-0.1%,-0.1%,-2.6%,0.2%
Washington,0 (0.0%),$-694 (-0.1%),-,"$-5,335 (-3.7%)",-0.8%,0.0%,-2.0%,0.0%
